In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.vision.all import *
from ipywidgets import widgets

In [ ]:
path = Path('../input/pnevmoniya/train')
fls = get_image_files(path)
failed = verify_images(path)
failed

In [ ]:
#DataBlock
pneumoniya = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.2,seed=70),
    get_y = parent_label,
    item_tfms = Resize(350)
)

#Dataloaders

dls = pneumoniya.dataloaders(path)

In [ ]:
dls.show_batch(max_n=34, nrows=4)

In [ ]:
#Teaching or Learning
learn = vision_learner(dls, resnet34, metrics = accuracy)
learn.fine_tune(5)

In [ ]:
interp=ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5,nrows=1)

In [ ]:
upload = widgets.FileUpload()
upload

In [ ]:
img = PILImage.create(upload.data[-1])
pred,pred_idx,probs = learn.predict(img)
print(f'Bashorat: {pred}\n'
     f'Ehtimollik: {probs[pred_idx]}')
img

In [ ]:
path_test = Path('../input/pnevmoniya/test')

In [ ]:
images = get_image_files(path_test)

In [ ]:
df = pd.read_csv('../input/pnevmoniya/sample_solution.csv')
df.head()

In [ ]:
pre = []
for image in images:
    pred, pred_idx, probs = learn.predict(image)
    name = image.name
    pre.append([image.name,pred])

In [ ]:
predicted = pd.DataFrame(pre)
predicted.head()

In [ ]:
for item in predicted:
    predicted = predicted.replace({1:{'PNEUMONIA':1,'NORMAL':0}})
predicted.head()

In [ ]:
predicted = predicted.sort_values(0)

In [ ]:
predicted

In [ ]:
predicted.index = predicted[0]

In [ ]:
predicted

In [ ]:
predicted.drop(0,axis=1)

In [ ]:
df.labels = predicted[1].values
df.head()

In [ ]:
df.to_csv('submission_to_pnevmation.csv',index=False)

In [ ]:
pd.read_csv('./submission_to_pnevmation.csv')